### Capstone Project -  E-Recruitment | S&P Global

### Sponsor:-  
 - Dattatrey, Surendra 
 
### Mentor:-  
 - Prof. S M Bendre

### Team Members:- 
    - Devender Bansal ID:- 11810089
    - Vikash Singh Negi-- 11810048
    - Sandeep Kumar Singh - 11810014
    - Atul Sharma - 11810126

In [208]:
import mammoth
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import urllib.request
import re
import os
import nltk, string
from nltk import word_tokenize 
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import subprocess
import os
import uuid
import pandas as pd
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 

In [96]:
path= "c://CBAWDR//rec//" 

In [97]:
docfiles=[]
pdffiles=[]
Unsupported=[]
def file_seg(s):
    flist=os.listdir(s)
    for file in flist:
        filename=file.split('.')
        if filename[1] == 'docx':
            docfiles.append(file)
        elif filename[1]=='pdf':
            pdffiles.append(file)
        else:
            msg="Unsupported format"
            Unsupported.append(msg)
        

In [98]:
def cleanpara(s):
    clean = re.compile('<.*?>')
    s = re.sub(clean, '', s)
    return s
   

In [99]:
paragraph=[]
def paras_doc(s):    
    for file in docfiles:
        new_path= path + str(file)
        with open(new_path, "rb") as doc_file:
            result = mammoth.convert_to_html(doc_file)
            html = result.value 
            soup = BeautifulSoup(html, 'html.parser')
            article=soup.text
            para= sent_tokenize(article)
            for par in para:
                cleanpara(par)
                paragraph.append(par)
    

In [100]:
def paras_pdf(s):
    for file in pdffiles:
        new_path= path + str(file)
        rsrcmgr = PDFResourceManager()
        retstr = BytesIO()
        codecs = 'utf-8'
        laparams = LAParams()
        device = HTMLConverter(rsrcmgr, retstr,codec=codecs,  laparams=laparams)
        fp = open(new_path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0 #is for all
        caching = True
        pagenos=set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        html = retstr.getvalue()
        retstr.close()
        soup = BeautifulSoup(html, 'html.parser')
        article=soup.text
        para= sent_tokenize(article)
        for par in para:
            cleanpara(par)
            paragraph.append(par)

In [101]:
paragraph=[]
def paragphs(s):
    file_seg(s)
    paras_doc(s)
    paras_pdf(s)
    para_df=pd.DataFrame(paragraph, columns=['Text'])
    para_df['Segment'] = pd.Series()

In [ ]:
JD_Para=[]
def jd_para(s):
    with open(s, "rb") as doc_file:
        result = mammoth.convert_to_html(doc_file)
        html = result.value 
        soup = BeautifulSoup(html, 'html.parser')
        article=soup.text
        para= sent_tokenize(article)
        for par in para:
            cleanpara(par)
            JD_Para(par)



In [102]:
paragphs(path)

In [213]:
data=pd.read_csv(r"c:\CBAWDR\Segments _Final.csv")

In [214]:
data.drop(['Unnamed: 3', 'Sn.No'] ,axis=1, inplace=True)

In [215]:
annotation_value=list(data['Segment'].drop_duplicates())
int_value=list(range(1,7))
annotation_id=dict(zip(annotation_value,int_value))
rev_annotation= dict(zip(int_value,annotation_value))
data['Id']=data['Segment'].map(annotation_id)
data=data.drop(columns=['Segment'])
data.dropna(how ='any', inplace=True)

In [216]:
#Tokenization, removal of stop words, removal of low frequency(3%) & High Frequency (97%) words. Encoding & creation of DTM
vectorizer=CountVectorizer(min_df = 0.03 , max_df= 0.97, stop_words='english')
vectorizer.fit(data.iloc[:,0])
dtm=vectorizer.transform(data.iloc[:,0])

In [217]:
#Defining Feature & label and splitting the data into training(70%) & testing(30%)
X,y=dtm,data.iloc[:,-1]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3)

In [218]:
#Train the classifier(Logistical Regression) with Training Data. Predicting the Label for test Data & Score the model on Accuracy
clf=LogisticRegression(random_state=0)
clf.fit(X_train,y_train)
y_predict=clf.predict(X_test)
score_Log=clf.score(X_test,y_test)
print(score_Log)

0.38461538461538464


In [219]:
resume_path="c://CBAWDR//resume//"

In [220]:
predict=[]
def dfm(list):
    df=pd.DataFrame(list, columns=['Text'])      
    label_para=vectorizer.transform(df.iloc[:,0])
    pred_label=clf.predict(label_para)
    predict.extend(pred_label)

In [221]:
docfiles=[]
pdffiles=[]
Unsupported=[]
paragraph=[]
paragphs(resume_path)
dfm(paragraph)
df_segment= pd.DataFrame(list(zip(paragraph, predict)), 
               columns =['Text', 'label']) 
rev_annotation= dict(zip(int_value,annotation_value))
df_segment['Segment']=df_segment['label'].map(rev_annotation)
df_segment.drop(['label'] ,axis=1, inplace=True)
segmentation=df_segment.groupby('Segment').agg({ 'Text': lambda x: ' '.join(x)}).reset_index()


In [ ]:
jd_predict=[]
def dfm(list):
    df=pd.DataFrame(list, columns=['Text'])      
    label_para=vectorizer.transform(df.iloc[:,0])
    pred_label=clf.predict(label_para)
    jd_predict.extend(pred_label)


In [ ]:
df_segment_jd= pd.DataFrame(list(zip(JD_Para, jd_predict)), 
               columns =['Text', 'label']) 
rev_annotation= dict(zip(int_value,annotation_value))
df_segment_jd['Segment']=df_segment_jd['label'].map(rev_annotation)
df_segment_jd.drop(['label'] ,axis=1, inplace=True)
segmentation_jd=df_segment_jd.groupby('Segment').agg({ 'Text': lambda x: ' '.join(x)}).reset_index()

In [ ]:
class color: 
UNDERLINE = '\033[4m'
END = '\033[0m'
k= segmentation_jd.shape[0]  
for i in range(k):
    print(color.BOLD + color.UNDERLINE + segmentation_jd.iloc[i,0] + color.END  + color.END)
    print(segmentation_jd.iloc[i,1])

In [231]:
class color: 
   UNDERLINE = '\033[4m'
   END = '\033[0m'
k= segmentation.shape[0]   

for i in range(k):
    print(color.BOLD + color.UNDERLINE + segmentation.iloc[i,0] + color.END  + color.END)
    print(segmentation.iloc[i,1])
    


Experience_Skills
Approx.Current Role	:	Senior Technical Account ManagerTotal year in Premier Business: 9+Contribution to Business Growth & CPECurrently Managing strategic high-visibility and high-touch customer like HCL, HP and Genpact Distinction of Managing Large number of customer in different verticals in different parts of India e.g. IT Roadmap & ProjectsIT OperationsIT GovernanceVendor Management Lifecycle Management for Company Portal & Website www.softwaredioxide.com  ,  www.qaiindia.com  Highlights Undertook & Completed IT projects on Capacity Enhancement. (Consultancy Management) from Birla Institute of Technology and Science (BITS), Pilani.Electronics and Telecommunication Engineer from the Institution of Electronics and Telecommunication Engineers, New Delhi.PERSONAL DETAILS  DATE OF BIRTH	:	29TH DEC. 1976.ADDRESS	:	C-201, Sanghamitra Apartments, Sector 4, Plot no.
Other
Devender BansalAlias: debansal, Ph. : 09873184914Total Work Experience	:	15 Yrs. Continuous Improvement

In [265]:
exp_data=segmentation.loc[(segmentation['Segment']== 'Experience') | (segmentation['Segment']== 'Experience_Skills')]
exp_list=exp_data['Text'].tolist()
exp_test=" ".join(exp_list)

In [266]:
print(exp_test)

Approx.Current Role	:	Senior Technical Account ManagerTotal year in Premier Business: 9+Contribution to Business Growth & CPECurrently Managing strategic high-visibility and high-touch customer like HCL, HP and Genpact Distinction of Managing Large number of customer in different verticals in different parts of India e.g. IT Roadmap & ProjectsIT OperationsIT GovernanceVendor Management Lifecycle Management for Company Portal & Website www.softwaredioxide.com  ,  www.qaiindia.com  Highlights Undertook & Completed IT projects on Capacity Enhancement. (Consultancy Management) from Birla Institute of Technology and Science (BITS), Pilani.Electronics and Telecommunication Engineer from the Institution of Electronics and Telecommunication Engineers, New Delhi.PERSONAL DETAILS  DATE OF BIRTH	:	29TH DEC. 1976.ADDRESS	:	C-201, Sanghamitra Apartments, Sector 4, Plot no.


In [ ]:
exp_text=exp_data.iloc[0,1] 

In [270]:
def exp_extractor(s):
    if re.search('experience', s):
        for sen in (s).split('.'):    
            if re.search('experience', sen):
                if re.search('overall | total', sen):
                    sen_tokenized=nltk.word_tokenize(sen)        
                    tagged=nltk.pos_tag(sen_tokenized)
                    entities = nltk.chunk.ne_chunk(tagged)
                    for subtree in entities.subtrees():
                        for leaf in subtree.leaves():
                            if leaf[1] =='CD':
                                experience=leaf[0]
                                if experience !="":                            
                                    tot_exp.append(experience)
                                else:                            
                                    tot_exp.append('Not Found')
    else:
        tot_exp.append('Not Found')
      
    

In [271]:
tot_exp=[]
exp_extractor(exp_test)


In [272]:
print(tot_exp)

['Not Found']


In [273]:
para_df_Resume=segmentation.loc[(segmentation['Segment']== 'Skills') | (segmentation['Segment']== 'Experience_Skills')]
para_df_JD=segmentation_jd.loc[(segmentation_jd['Segment']== 'Skills') | (segmentation_jd['Segment']== 'Experience_Skills')]

In [ ]:
#### NLTK work and sentence tokinizer
skill_level=[]
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
para_df_Resume['Segments'] = para_df_Resume.Text.map(sent_tokenize)
para_df_Resume['tokens_sentences'] = para_df_Resume['Segments'].map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
para_df_JD['Segments'] = para_df_JD.Text.map(sent_tokenize)
para_df_JD['tokens_sentences'] = para_df_JD['Segments'].map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])

para_df_Resume['tokens_sentences']
para_df_JD['tokens_sentences']

## NLTK postagging

from nltk import pos_tag
para_df_Resume['POS_tokens'] = para_df_Resume['tokens_sentences'].map(lambda tokens_sentences: 
                                                           [pos_tag(tokens) for tokens in tokens_sentences])
para_df_JD['POS_tokens'] = para_df_JD['tokens_sentences'].map(lambda tokens_sentences: 
                                                           [pos_tag(tokens) for tokens in tokens_sentences])

#identifying only nouns
# for sentence in data['articles']:
###     data['nouns'] = [token for token, pos in pos_tag(word_tokenize(sentence)) if pos.startswith('N')]

## NLTK lemmatizing and pos tagging
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatizing each word with its POS tag, in each sentence
para_df_Resume['tokens_sentences_lemmatized'] = para_df_Resume['POS_tokens'].map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

para_df_JD['tokens_sentences_lemmatized'] = para_df_JD['POS_tokens'].map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

# stop words
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
#stopwords_other = ['one', 'mr', 'bbc', 'image', 'getty', 'de', 'en', 'caption', 'also', 'copyright', 'something']
my_stopwords = stopwords.words('English') #+ stopwords_verbs + stopwords_other

from itertools import chain
para_df_Resume['tokens'] = para_df_Resume['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
para_df_Resume['tokens'] = para_df_Resume['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>2])

para_df_JD['tokens'] = para_df_JD['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
para_df_JD['tokens'] = para_df_JD['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>2])


para_df_Resume['tokens']

## looking for by grams and tri grams

from gensim.models import Phrases
nouns= []
tokens = para_df_Resume['tokens'].tolist()
# for tok in tokens:
#     #print(pos_tag(tok))
#     nouns.append([token for token, pos in pos_tag(tok) if pos.startswith('N')])
# nouns = list(filter(None, nouns))
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])
tokens = list(filter(None, tokens))


tokens1 = para_df_JD['tokens'].tolist()
# for tok in tokens:
#     #print(pos_tag(tok))
#     nouns.append([token for token, pos in pos_tag(tok) if pos.startswith('N')])
# nouns = list(filter(None, nouns))
bigram_model1 = Phrases(tokens1)
trigram_model1 = Phrases(bigram_model[tokens1], min_count=1)
tokens1 = list(trigram_model[bigram_model[tokens1]])
tokens1 = list(filter(None, tokens1))

tokens1

#### unique words list
unique_list = [] 
for x in tokens:
    for y in x:
        if y not in unique_list: 
            unique_list.append(y)
unique_list = [w.replace('_', '') for w in unique_list]
len(unique_list)


#JD
unique_listJD = [] 
for x in tokens1:
    for y in x:
        if y not in unique_listJD: 
            unique_listJD.append(y)
unique_listJD = [w.replace('_', '') for w in unique_listJD]
len(unique_listJD)

def chk_lst(wrd,lst):
    flg=False
    for i in lst:
        if re.search(wrd,i):
            flg=True
            return flg
    return flg
        

def getmetrics(inp, result):
    # Regexp match with any string in the list
#     tn = len([value for value in inp if chk_lst(value,result)])
    tp = len([value for value in inp if chk_lst(value,result)])
    fp = len([value for value in inp if not chk_lst(value,result)])
    fn = len([value for value in result if not chk_lst(value,inp)])
    tp_val = [value for value in inp if chk_lst(value,result)]
    print("True Positives - %s, False Positives - %s, False Negatives - %s"%(tp,fp,fn))
    if tp>0:
#         accuracy = (tp+tn)/(tp+fp+fn+tn)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1 = (2*precision*recall)/(precision+recall)
    else:
#         accuracy=0
        precision=0
        recall = 0
        f1 = 0
    return precision,recall,f1,tp_val

# Evaluation of Only nltk Tokens - Model 1
unique_list_low = [i.lower() for i in unique_list]
unique_listJD_low = [i.lower() for i in unique_listJD]
m1_precision, m1_recall, m1_f1, mat_val_m1 = getmetrics(unique_listJD_low, unique_list_low)
print("Model 1 Precision - %s \nModel 1 Recall - %s \nModel 1 F1 - %s"%( m1_precision,m1_recall,m1_f1))

if (m1_precision >= 0.3 and float(m1_f1) >= 0.3) :
    print("Best")
    skill_level.append('High')
elif (m1_precision >= 0.2 and float(m1_f1) >= 0.2) :
    print("Good")
    skill_level.append('Moderate')
else:
    print("Nakli")
    skill_level.append('Low')



In [276]:
para_df_Resume_exp=exp_data.copy()

In [274]:

Rel_Exp=[]
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
para_df_Resume_exp['Segments'] = para_df_Resume_exp.Text.map(sent_tokenize)
para_df_Resume_exp['tokens_sentences'] = para_df_Resume_exp['Segments'].map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
para_df_JD['Segments'] = para_df_JD.Text.map(sent_tokenize)
para_df_JD['tokens_sentences'] = para_df_JD['Segments'].map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])

para_df_Resume_exp['tokens_sentences']
para_df_JD['tokens_sentences']

## NLTK postagging

from nltk import pos_tag
para_df_Resume_exp['POS_tokens'] = para_df_Resume_exp['tokens_sentences'].map(lambda tokens_sentences: 
                                                           [pos_tag(tokens) for tokens in tokens_sentences])
para_df_JD['POS_tokens'] = para_df_JD['tokens_sentences'].map(lambda tokens_sentences: 
                                                           [pos_tag(tokens) for tokens in tokens_sentences])

#identifying only nouns
# for sentence in data['articles']:
###     data['nouns'] = [token for token, pos in pos_tag(word_tokenize(sentence)) if pos.startswith('N')]

## NLTK lemmatizing and pos tagging
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatizing each word with its POS tag, in each sentence
para_df_Resume_exp['tokens_sentences_lemmatized'] = para_df_Resume_exp['POS_tokens'].map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

para_df_JD['tokens_sentences_lemmatized'] = para_df_JD['POS_tokens'].map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

# stop words
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
#stopwords_other = ['one', 'mr', 'bbc', 'image', 'getty', 'de', 'en', 'caption', 'also', 'copyright', 'something']
my_stopwords = stopwords.words('English') #+ stopwords_verbs + stopwords_other

from itertools import chain
para_df_Resume_exp['tokens'] = para_df_Resume_exp['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
para_df_Resume_exp['tokens'] = para_df_Resume_exp['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>2])

para_df_JD['tokens'] = para_df_JD['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
para_df_JD['tokens'] = para_df_JD['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>2])


para_df_Resume_exp['tokens']

## looking for by grams and tri grams

from gensim.models import Phrases
nouns= []
tokens = para_df_Resume_exp['tokens'].tolist()
# for tok in tokens:
#     #print(pos_tag(tok))
#     nouns.append([token for token, pos in pos_tag(tok) if pos.startswith('N')])
# nouns = list(filter(None, nouns))
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])
tokens = list(filter(None, tokens))


tokens1 = para_df_JD['tokens'].tolist()
# for tok in tokens:
#     #print(pos_tag(tok))
#     nouns.append([token for token, pos in pos_tag(tok) if pos.startswith('N')])
# nouns = list(filter(None, nouns))
bigram_model1 = Phrases(tokens1)
trigram_model1 = Phrases(bigram_model[tokens1], min_count=1)
tokens1 = list(trigram_model[bigram_model[tokens1]])
tokens1 = list(filter(None, tokens1))

tokens1

#### unique words list
unique_list = [] 
for x in tokens:
    for y in x:
        if y not in unique_list: 
            unique_list.append(y)
unique_list = [w.replace('_', '') for w in unique_list]
len(unique_list)


#JD
unique_listJD = [] 
for x in tokens1:
    for y in x:
        if y not in unique_listJD: 
            unique_listJD.append(y)
unique_listJD = [w.replace('_', '') for w in unique_listJD]
len(unique_listJD)

def chk_lst(wrd,lst):
    flg=False
    for i in lst:
        if re.search(wrd,i):
            flg=True
            return flg
    return flg
        

def getmetrics(inp, result):
    # Regexp match with any string in the list
#     tn = len([value for value in inp if chk_lst(value,result)])
    tp = len([value for value in inp if chk_lst(value,result)])
    fp = len([value for value in inp if not chk_lst(value,result)])
    fn = len([value for value in result if not chk_lst(value,inp)])
    tp_val = [value for value in inp if chk_lst(value,result)]
    print("True Positives - %s, False Positives - %s, False Negatives - %s"%(tp,fp,fn))
    if tp>0:
#         accuracy = (tp+tn)/(tp+fp+fn+tn)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1 = (2*precision*recall)/(precision+recall)
    else:
#         accuracy=0
        precision=0
        recall = 0
        f1 = 0
    return precision,recall,f1,tp_val

# Evaluation of Only nltk Tokens - Model 1
unique_list_low = [i.lower() for i in unique_list]
unique_listJD_low = [i.lower() for i in unique_listJD]
m1_precision, m1_recall, m1_f1, mat_val_m1 = getmetrics(unique_listJD_low, unique_list_low)
print("Model 1 Precision - %s \nModel 1 Recall - %s \nModel 1 F1 - %s"%( m1_precision,m1_recall,m1_f1))

if (m1_precision >= 0.3 and float(m1_f1) >= 0.3) :
    Print("Best")
    Rel_Exp.append('High')
elif (m1_precision >= 0.2 and float(m1_f1) >= 0.2) :
    print("Good")
    Rel_Exp.append('Moderate')
else:
    print("Nakli")
    Rel_Exp.append('Low')




In [ ]:
masters_list = ['pgdm','mba','msc','md','md','mca']
batchelors_list = ['bsc','bba','ba','be','btech','barch']

tier1 = file['university_name'].tolist()
tier1


tok = tokens


degree = []
for i in tok:
    if set(masters_list).intersection(i) != set():
        degree.append('masters')
    elif set(batchelors_list).intersection(i) != set():
        degree.append('bachelors')
    else:
        degree.append("others")
        
        
        
mystring = 'university of delhi'
s = mystring.replace(" ", "_")
s


tier1_list = []
for i in tier1:
    s = i.replace(" ","_")
    tier1_list.append(s.lower())
print(tier1_list)



tier1_match = []
for i in tok:
    if set(tier1_list).intersection(i) != set():
        tier1_match.append('masters')
    elif set(tier1_list).intersection(i) != set():
        tier1_match.append('bachelors')
    else:
        tier1_match.append("others")
        
        
tier1india = pd.read_csv('tier1india.csv')
tier2india = pd.read_csv('tier2india.csv')
tier3india = pd.read_csv('tier3india.csv')

tier1india_list = tier1india['univ'].tolist()
tier2india_list = tier2india['univ'].tolist()
tier3india_list = tier3india['univ'].tolist()


tier1_list_india = []
for i in tier1india_list:
    s = i.replace(" ","_")
    tier1_list_india.append(s.lower())
print(tier1_list_india)

tier2_list_india = []
for i in tier2india_list:
    s = i.replace(" ","_")
    tier2_list_india.append(s.lower())
print(tier2_list_india)


tier3_list_india = []
for i in tier3india_list:
    s = i.replace(" ","_")
    tier3_list_india.append(s.lower())
print(tier3_list_india)


tier_india_match = []
for i in tok:
    if set(tier1_list_india).intersection(i) != set():
        tier_india_match.append('Tier1')
    elif set(tier2_list_india).intersection(i) != set():
        tier_india_match.append('Tier2')
    elif set(tier3_list_india).intersection(i) != set():
        tier_india_match.append('Tier3')
    else:
        tier_india_match.append("others")

In [ ]:
data=pd.DataFrame(Resume_Names, columns =['ResumeName'])

data['Total Experience']=tot_exp
data['Relevant Experience']=Rel_Exp
data['Skill Level']=skill_level
data['College Tier']=tier
data['Degree']=degree


In [ ]:
def converter(s):
    if s=='High':
        return 2
    elif s=='Moderate':
        return 1
    else:
        return 0
    
def converter_tier(s):
    if s=='Tier 1':
        return 2
    elif s=='Tier 2':
        return 1
    else:
        return 0
def converter_edu(s):
    if s=='Masters':
        return 2
    elif s=='Bachelors':
        return 1
    else:
        return 0

data['Relevant_Experience']=data['Relevant Experience'].apply(converter)
data['Skill_Level']=data['Skill Level'].apply(converter)
data['College_Tier']=data['College Tier'].apply(converter_tier)
data['degree']=data['Degree'].apply(converter_edu)

feature_cols = ['Total Experience', 'Relevant_Experience', 'Skill_Level', 'College_Tier','degree',]
X = data[feature_cols] # Features
y = data.Label # Target variable

#Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test


# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


#Predict the response for theEW dataset
new_pred = clf.predict(X_new)

print(y_pred)
print(X_test)
print(X_new)



for i in range(len(new_pred)):
    if new_pred[i]==1:
        print('The Candidate is Accepted')
    else:
        print('The Candidate is Rejected')


In [ ]:
The Candidate is Rejected
The Candidate is Rejected
The Candidate is Rejected
The Candidate is Accepted
The Candidate is Rejected
The Candidate is Rejected
The Candidate is Rejected
The Candidate is Accepted
The Candidate is Rejected
The Candidate is Rejected
The Candidate is Rejected